#### Ray Tracer code for liquid xenon TPC studies.  Erik Hogenbirk, Nikhef Amsterdam, Februari 2015

###### For questions, bugs, feedback, etc., mail me at ehogenbi@nikhef.nl

In [ ]:
#import pygame
#import sys
import random
import math
import string
from math import sin, cos, atan, asin, acos, atan2
from numpy import array, float64, zeros
from numpy.linalg import norm
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from numpy import arange
import matplotlib.pyplot as plt
#%matplotlib inline
import pickle
import time
from tqdm import tqdm

In [ ]:
# Initialization

# Geometry
rTpc = 22.5     # Value in mm, radius of tpc
zPmt1 = 0.      # z-positions, measured from bottom PMT
zBot = 4.       # Positions from drawing assuming PMT1 at 37.5 on that scale, not corrected for shrinking
zCat = 21.
zGat = 120.1
zLiq = 122.6
zAno = 125.2
zTop = 130.3
zPmt2 = 134.3   # Assuming 4 mm spacing mesh-PMT
zMeshes = [zBot,zCat,zGat,zAno,zTop]

# Other physical parameters
absProbMesh = 0.117 
absProbWall = 0.1
nLxe = 1.69                      # http://arxiv.org/ftp/physics/papers/0307/0307044.pdf . Critical angle of 36 degrees

# Starting point
xIni = 10.
yIni = 0.
zIni = 50.



In [ ]:
# Auxilary functions
def spherical_to_cartesian(spherical_vect, cart_vect):
    """Convert the spherical coordinate vector [r, theta, phi] to the Cartesian vector [x, y, z].

    The parameter r is the radial distance, theta is the polar angle, and phi is the azimuth.
    
    @param spherical_vect:  The spherical coordinate vector [r, theta, phi].
    @type spherical_vect:   3D array or list
    @param cart_vect:       The Cartesian vector [x, y, z].
    @type cart_vect:        3D array or list
    """

    # Trig alias.
    sin_theta = sin(spherical_vect[1])

    # The vector.
    cart_vect[0] = spherical_vect[0] * cos(spherical_vect[2]) * sin_theta
    cart_vect[1] = spherical_vect[0] * sin(spherical_vect[2]) * sin_theta
    cart_vect[2] = spherical_vect[0] * cos(spherical_vect[1])
    # http://en.wikipedia.org/wiki/Spherical_coordinate_system#mediaviewer/File:3D_Spherical.svg

def cartesian_to_spherical(vector):
    """Convert the Cartesian vector [x, y, z] to spherical coordinates [r, theta, phi].

    The parameter r is the radial distance, theta is the polar angle, and phi is the azimuth.


    @param vector:  The Cartesian vector [x, y, z].
    @type vector:   numpy rank-1, 3D array
    @return:        The spherical coordinate vector [r, theta, phi].
    @rtype:         numpy rank-1, 3D array
    """

    # The radial distance.
    r = norm(vector)
    # Unit vector.
    unit = vector / r
    # The polar angle.
    theta = acos(unit[2])
    # The azimuth.
    phi = atan2(unit[1], unit[0])
    # Return the spherical coordinate vector.
    return array([r, theta, phi], float64)

def getReflectance(n1,th1,n2,th2):
    # Function to get reflection probability at a surface between two materials with different index of refraction
    # Note that th2 is the transmitted angle
    # These formulas came from the best source ever wikipedia but I also saw this stuff in Hecht
    # There are two polarization, but assuming the photons are not polarized it's just the average
    Rs = ((n1*cos(th1)-n2*cos(th2))/(n1*cos(th1)+n2*cos(th2)))**2
    Rp = ((n1*cos(th2)-n2*cos(th1))/(n1*cos(th2)+n2*cos(th1)))**2
    R = 0.5*(Rs+Rp)
    return R

def getSpecularReflectionProbability(incidentAngle):
    # get the probability of specular reflection at lXe - Teflon interface
    # Source: http://repositorium.uni-muenster.de/document/miami/42495273-2cc7-4632-99af-accf074921e7/diss_levy.pdf page 225 (250)
    incidentAngle = abs(math.degrees(incidentAngle))
    if   incidentAngle <= 47.5 : return 0.
    elif incidentAngle <= 52.5 : return 0.03
    elif incidentAngle <= 57.5 : return 0.05
    elif incidentAngle <= 62.5 : return 0.19
    elif incidentAngle <= 67.5 : return 0.46
    elif incidentAngle <= 72.5 : return 0.87
    elif incidentAngle  > 72.5 : return 1.

def getDiffuseReflectionProbability(incidentAngle):
    # get the probability of diffuse reflection at lXe - Teflon interface
    # Source: http://repositorium.uni-muenster.de/document/miami/42495273-2cc7-4632-99af-accf074921e7/diss_levy.pdf page 225 (250)
    incidentAngle = abs(math.degrees(incidentAngle))
    
    if   incidentAngle <= 37.5 : return 0.31
    elif incidentAngle <= 42.5 : return 0.33
    elif incidentAngle <= 47.5 : return 0.31
    elif incidentAngle <= 52.5 : return 0.40
    elif incidentAngle <= 57.5 : return 0.29
    elif incidentAngle <= 62.5 : return 0.22
    elif incidentAngle <= 67.5 : return 0.

    
    
#    return np.piecewise(incidentAngle,
#                     [incidentAngle <= 47.5,incidentAngle <= 52.5,incidentAngle <= 57.5,incidentAngle <=62.5,incidentAngle <=67.5,
#                      incidentAngle <= 72.5, incidentAngle>72.5],
#                     [0.,0.03,0.05,0.19,0.46,0.87,1.])

def plotReflectionProbabilities():
    # Plot the specular and diffuse reflection probability as a function of incident angle
    x  = [angle for angle in arange(0.,math.pi/2,math.pi/2*0.01)]
    y1 = [getSpecularReflectionProbability(angle) for angle in x]
    y2 = [getDiffuseReflectionProbability(angle) for angle in x]
    plt.plot(x,y1,label='Specular')
    plt.plot(x,y2,label='Diffuse')
    plt.legend()
    plt.show()    

In [ ]:
# Simulation parameters
nPh  = 10               # Simulate nph photons
step = 0.2              # Step size in mm. Keep this small with respect to the smallest tpc dimensions; i.e. mesh separation, wall curvature, etc.
# Reflection types
#reflectionType = "specular"    
reflectionType = "diffuse"
#reflectionType = "combined"
# you can turn on/off text output, tracing of the photons and saving the metadatasets list to file
outputText  = False
tracing     = True
saveToFile  = False
outFilePath = 'test.pkl'

# List containing lists of particle position
if tracing : 
    xTraces = []
    yTraces = []
    zTraces = []

# This is the list of libraries where all the data about the photons is stored, see 'metadata'
metadatasets = []    
    
# Loop over grid points
# tqdm will keep track of the progress here
#for zIni in tqdm(arange(0.,135.,1.)):
for zIni in arange(50.,51.,2.): #arange(21.,119.,2.):
    for xIni in arange(0.,1.,2.):
    #for xIni in arange(0.,23.,1.): #arange(0.,22.,2.):
        # All you ever wanted to know about the photons. Mainly how they 'die'
        metadata  = {'xIni':xIni,'yIni':yIni,'zIni':zIni,'nPhotons': 0, 'PMT1': 0 , 'PMT2':0 , 'Mesh':0,  'Wall':0}

        for i in range(0,nPh): 
            # Give current photon number
            if outputText : print("============= Photon number %d =============" % i)
            # We keep track of the total distance covered by the photon
            distanceCovered = 0.

            # Initialize (or reset) the list where the particle paths are stored
            if tracing : 
                xTrace = []
                yTrace = []
                zTrace = []                
                
            # Start of the photon at initial position
            xPh = xIni
            yPh = yIni
            zPh = zIni

            # Starting direction is random;
            cosThPh = random.uniform(-1,1)
            thPh = np.arccos(cosThPh)
            phPh = random.uniform(0, 2*math.pi)
            # Transform the spherical coordinates to cathesian for easier evolution
            normal = [0,0,0]
            spherical_to_cartesian([1.,thPh,phPh],normal)

            photonlives=True
            while photonlives:
                # Add step size to the distance covered
                distanceCovered += step
                # Get new position
                xPhnew = xPh + normal[0]*step
                yPhnew = yPh + normal[1]*step
                zPhnew = zPh + normal[2]*step

                # Check if one of the meshes is passed
                # Condition: z of mesh betweeen old and new position (two options for going upwards or downwards)
                for zMesh in zMeshes:
                    if (zPh < zMesh and zPhnew > zMesh) or (zPh > zMesh and zPhnew < zMesh):
                        if outputText : print("Mesh passed.")
                        if random.random() < absProbMesh :
                            if outputText : print("Photon died at mesh :(")
                            metadata['Mesh']+=1
                            photonlives = False

                # Check if photon goes into one of the PMTs
                if zPhnew<zPmt1 :
                    if outputText : print("PMT1 captured the photon!")
                    metadata['PMT1']+=1
                    photonlives = False
                if zPhnew>zPmt2 :
                    if outputText : print("PMT2 captured the photon!")
                    metadata['PMT2']+=1
                    photonlives = False

                # Check if we crossed the liquid-gas iterface
                # NOTE for now, this is just going to the gas, back into the liquid is neglected
                if zPhnew>zLiq and zPh < zLiq :
                    # Now it gets interesting. First we get the incident angle.
                    sphericalvector = cartesian_to_spherical(normal)
                    thinc = sphericalvector[1]
                    if outputText : print("Encoutered liquid-gas surface, incident angle %f degrees" % math.degrees(thinc))
                    thcrit = asin(1./nLxe)
                    if thinc > thcrit:
                        sphericalvector[1] = math.pi - thinc
                        spherical_to_cartesian(sphericalvector,normal)
                        if outputText : print("Total reflection.")
                    else:
                        # If there is a smaller angle, there is a probability of transmission or reflection
                        thtr = asin(nLxe*sin(thinc))
                        reflectionprob = getReflectance(nLxe,thinc,1.,thtr)
                        if random.random() < reflectionprob:
                            # Reflect anyway (with this probability)
                            sphericalvector[1] = math.pi - thinc
                            spherical_to_cartesian(sphericalvector,normal)
                            if outputText : print("Reflection.")
                        else:
                            # Transmission!
                            sphericalvector[1] = thtr
                            spherical_to_cartesian(sphericalvector,normal)
                            if outputText : print("Transmission.")



                # Check if the wall is reached
                if xPhnew*xPhnew+yPhnew*yPhnew > rTpc*rTpc:
                    # If the photon reaches a wall, we must reflect it
                    # We first get the angle in the x-y plane normal to the wall (i.e. towards the centre)
                    if xPhnew<0:
                        phiNormal = atan(yPhnew/xPhnew)
                    if xPhnew>0:
                        phiNormal = atan(yPhnew/xPhnew)+math.pi
                    # Get the current direction of the photon in spherical coordinates
                    sphericalvector = cartesian_to_spherical(normal)
                    # The incident angle is the normal vector angle minus the original angle. Note that the photon travels TO the wall (i.e. include an extra 180 degreess)
                    # NOTE this is just in the xy-plane
                    incidentAngle = phiNormal - (sphericalvector[2] + math.pi)
                    if outputText : 
                        print("Wall reached at x= %f, y = %f,z = %f, incident angle %f degrees." % (
                            xPhnew, yPhnew,zPhnew,math.degrees(incidentAngle)))
                    # We can set different types of reflection.
                    if reflectionType == "combined":
                        randomNumber = random.random()
                        # NOTE this doesn't work yet! have to convert to degrees and get in range 0 to 90 degrees
                        # oh and the incident angle should probably be in the incident plane...
                        # http://math.stackexchange.com/questions/11346/how-to-compute-the-angle-between-two-vectors-expressed-in-the-spherical-coordina
                        # http://math.stackexchange.com/questions/243142/what-is-the-general-formula-for-calculating-dot-and-cross-products-in-spherical
                        incidentAnglePlane = acos(-(sin(sphericalvector[2])*sin(phiNormal)*cos(sphericalvector[1]-math.pi*0.5)+cos(sphericalvector[2])*cos(phiNormal)))
                        if outputText : print("Incident angle in plane of incidence is %f degrees." % math.degrees(incidentAnglePlane))
                        specProb = getSpecularReflectionProbability(incidentAnglePlane)
                        diffProb = getSpecularReflectionProbability(incidentAnglePlane)
                        if   randomNumber < specProb            : useReflection = "specular"
                        elif randomNumber < specProb + diffProb : useReflection = "diffuse"
                        else                                    : useReflection = "kill"
                    elif reflectionType == "diffuse" : useReflection = "diffuse"
                    elif reflectionType == "specular": useReflection = "specular"
                    if useReflection == "diffuse":
                        # We accept any angle at max 90 degrees from the normal.
                        # Azimuthal angle theta is random
                        # All directions equally likely.
                        sphericalvector = cartesian_to_spherical(normal)
                        cosThPh = random.uniform(-1,1)
                        sphericalvector[1] = np.arccos(cosThPh)
                        sphericalvector[2] = random.uniform(phiNormal-math.pi*0.5,phiNormal+math.pi*0.5)
                    if useReflection == "specular":
                        # For specular, we keep the azimuth unchanged and add the normal angle to the incident angle
                        sphericalvector[2] = phiNormal+incidentAngle
                    normal = [0,0,0]
                    spherical_to_cartesian(sphericalvector,normal)
                    # Now, we have to put the photon right ON the wall, because otherwise we might travel through the wall
                    rprime = math.sqrt(xPhnew**2+yPhnew**2)
                    xPhnew = xPhnew*rTpc/rprime
                    yPhnew = yPhnew*rTpc/rprime
                    if (random.random()<absProbWall and reflectionType != "combined") or useReflection=="kill":
                        if outputText : print("Photon died at wall :(")
                        metadata['Wall']+=1
                        photonlives = False

                # Change new position to 'old' postion for next loop
                xPh = xPhnew
                yPh = yPhnew
                zPh = zPhnew

                # Store path in xTrace, etc
                if tracing : 
                    xTrace.append(xPh)
                    yTrace.append(yPh)
                    zTrace.append(zPh)
            # The following is outside the photon loop. This means that a photon has just died.
            # We get the distance covered back in mm
            if outputText : print("Distance covered: %.1f mm." % (distanceCovered) )
            if tracing : 
                xTraces.append(xTrace)
                yTraces.append(yTrace)
                zTraces.append(zTrace)
            metadata['nPhotons']+=1
        metadatasets.append(metadata)


if saveToFile:
    outputFile = open(outFilePath, 'wb')
    pickle.dump(metadatasets, outputFile)
    outputFile.close()
# Syntax to get it back:
# pickle.load( open( "save.p", "rb" ) 

print("Done!")

In [ ]:
# Plot the last photon path in 3D
mpl.rcParams['legend.fontsize'] = 10
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot(xTrace, yTrace, zTrace, label='Photon path')
ax.legend()
plt.show()

In [ ]:
# Plot the photon paths in 3D
mpl.rcParams['legend.fontsize'] = 10
fig = plt.figure()
ax = fig.gca(projection='3d')
for i in range(10):
    ax.plot(xTraces[i], yTraces[i], zTraces[i])
ax.legend()
plt.show()

In [ ]:
# Short check that the initial directions are uniform
mpl.rcParams['legend.fontsize'] = 10
fig = plt.figure()
ax = fig.gca(projection='3d')
xs = []
ys = []
zs = []
for i in range(0,3000):
    cosThPh = random.uniform(-1,1)
    thPh = np.arccos(cosThPh)
    phPh = random.uniform(0, 2*math.pi)
    normal = [0,0,0]
    spherical_to_cartesian([1.,thPh,phPh],normal)
    xs.append(normal[0])
    ys.append(normal[1])
    zs.append(normal[2])
ax.scatter(xs,ys,zs,depthshade=False)
ax.legend()
plt.show()


In [ ]:
xlist    = [metadatasets[i]['xIni'] for i in range(len(metadatasets))]
zlist    = [metadatasets[i]['zIni'] for i in range(len(metadatasets))]
pmt1list = [metadatasets[i]['PMT1']/metadatasets[i]['nPhotons']  for i in range(len(metadatasets))]
pmt2list = [metadatasets[i]['PMT2']/metadatasets[i]['nPhotons']  for i in range(len(metadatasets))]
meshlist = [metadatasets[i]['Mesh']/metadatasets[i]['nPhotons']  for i in range(len(metadatasets))]
walllist = [metadatasets[i]['Wall']/metadatasets[i]['nPhotons']  for i in range(len(metadatasets))]


In [ ]:
plt.plot(zlist,pmt1list,label='PMT1')
plt.plot(zlist,pmt2list,label='PMT2')
plt.plot(zlist,meshlist,label='Mesh')
plt.plot(zlist,walllist,label='Wall')
mpl.rcParams['legend.fontsize'] = 15
plt.legend()
plt.show()

In [ ]:
metadatasets

In [ ]:
# Plot the reflectance vs the angle in radians
thetalist = [theta for theta in arange(0.,0.63318,0.0001)] # Need high res for the last steep part
Rlist = [getReflectance(1.69,theta,1.,asin(nLxe*sin(theta))) for theta in thetalist]
plt.plot(thetalist,Rlist)
plt.show()

In [ ]:
metadatasets = pickle.load( open( "Diffuse_grid1mm_gooddim_10000ph_step02mm.pkl", "rb" ) )

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
from matplotlib.mlab import bivariate_normal

xSteps = 23
zSteps = 135

xArray    = array([[metadatasets[i*xSteps+j]['xIni'] for j in range(xSteps) ] for i in range(zSteps)])
zArray    = array([[metadatasets[i*xSteps+j]['zIni'] for j in range(xSteps) ] for i in range(zSteps)])
pmt1Array = array([[metadatasets[i*xSteps+j]['PMT1'] for j in range(xSteps) ] for i in range(zSteps)])

plt.plot()
plt.pcolor(xArray, zArray, pmt1Array,  cmap='PuBu_r') # norm=LogNorm(vmin=Z1.min(), vmax=Z1.max()),
plt.colorbar()

plt.show()